In [1]:
import hyperopt
from hyperopt import fmin, tpe, hp, space_eval

import pandas as pd
import numpy as np
from pyids import IDS
from pyids.algorithms import mine_CARs
from pyids.algorithms.ids_multiclass import IDSOneVsAll
from pyids.data_structures import IDSRuleSet

from pyarc.qcba.data_structures import QuantitativeDataFrame
from pyarc.data_structures import TransactionDB
from pyarc import CBA

import random
import logging
import time

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df = pd.read_csv("../data/iris0.csv")
quant_dataframe = QuantitativeDataFrame(df)


In [3]:

ids_multiclass = IDSOneVsAll(algorithm="RUSM")
ids_multiclass.fit(quant_dataframe, lambda_array=[1, 1, 0, 0, 100000, 10000, 1000000], rule_cutoff=200)
auc = ids_multiclass.score_auc(quant_dataframe)



In [4]:
def is_solution_interpretable(metrics):
    return (
        metrics["fraction_overlap"] <= 0.10 and
        metrics["fraction_classes"] == 1.0 and
        metrics["fraction_uncovered"] <= 0.15 and
        metrics["average_rule_width"] < 8 and
        metrics["ruleset_length"] <= 10
    )

In [5]:
def objective(args):
    lambda_array = list(args.values())
    print(lambda_array)
    
    ids_multiclass = IDSOneVsAll(algorithm="RUSM")
    ids_multiclass.fit(quant_dataframe, lambda_array=lambda_array, rule_cutoff=30)
    
    metrics = ids_multiclass.score_interpretability_metrics(quant_dataframe)
    
    if not is_solution_interpretable(metrics):
        return 0
    
    auc = ids_multiclass.score_auc(quant_dataframe)
    print("AUC", auc)

    return -auc

space = {
    "lambda1": hp.uniform("l1", 0, 1000),
    "lambda2": hp.uniform("l2", 0, 10000000),
    "lambda3": hp.uniform("l3", 0, 500),
    "lambda4": hp.uniform("l4", 0, 500),
    "lambda5": hp.uniform("l5", 0, 10000000),
    "lambda6": hp.uniform("l6", 0, 10000000),
    "lambda7": hp.uniform("l7", 0, 10000000)
}

best = fmin(objective, space, algo=tpe.suggest, max_evals=500)

print(best)

[292.40542823258585, 8558250.548718462, 8.533490940484, 135.69820408402106, 7301306.304354309, 585465.6252552149, 741571.0589130553]
  0%|                                                                          | 0/500 [00:00<?, ?trial/s, best loss=?]

D:\programy\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)

D:\programy\Anaconda\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



[210.35152142869327, 5605702.827383794, 372.48857826673265, 231.26660692287084, 4582144.260252676, 2283808.7438924527, 6482859.685696906]
[901.9783083466112, 1933613.1905032129, 477.28740142398846, 254.57561389473915, 6962877.353924441, 3041733.5613727192, 8759535.63126139]
[498.3469601781455, 5985058.206348775, 274.2015076689583, 428.19220363165016, 8416011.710204463, 3991191.2380608083, 2389703.8917295076]
[443.3342569825347, 8231990.9365935065, 487.7841341692449, 301.2300307058133, 4393479.11484199, 6633727.2242137715, 4613333.465376459]
[323.6479631255702, 1654043.078122418, 51.8356636254021, 413.4509111222296, 7876217.435816454, 6702244.502650456, 3553247.0251272386]
[418.7853535248185, 9589903.410622295, 180.31495973574025, 75.2259785488702, 8974218.021353954, 5989009.763281103, 1688089.616587809]
[994.2517901493234, 1533624.3575885245, 429.98165116767274, 177.194602876156, 8710352.984735278, 9196078.459551906, 2936197.9184338106]
[335.9942855751555, 3511276.0145960567, 351.08455

D:\programy\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)

D:\programy\Anaconda\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



[42.134716606060266, 4748439.877859656, 320.2218500793018, 130.5558872788932, 332499.0932336361, 113349.45767442742, 6133949.151933845]
[265.2726198799679, 8991080.185983514, 225.2620019912568, 136.06899153570876, 3621199.1389952167, 1014061.5266011065, 7232749.8071969515]
[869.5903267939296, 89716.81458992604, 97.1731142480796, 12.384636813567639, 5666065.2915170565, 12403.955646035261, 8434903.385405613]
AUC                                                                                                                    
0.9611111111111112                                                                                                     
[837.8011256646976, 476016.2147081923, 122.84567476358258, 5.24789119572651, 1388795.9577555675, 1999202.2689391675, 9947369.04830456]
[986.9698764101963, 2384579.148182185, 84.39365699213295, 29.36160864232567, 5662756.644828149, 108413.42552242824, 8229273.623690988]
AUC                                                                             

...

[493.3829152925646, 5908256.861352971, 208.88635952666863, 189.42265023502884, 8112202.835424853, 1450695.773190093, 9300062.385576963]
[614.5438443629673, 4120800.9200376105, 54.26812092055967, 173.63799417171458, 9613736.880847018, 14075.743166531494, 7688404.73783168]
AUC                                                                                                                    
0.9611111111111112                                                                                                     
[95.71747398262566, 6612443.874432099, 70.09388327303708, 485.80366754886484, 9532667.391018378, 11724.742464369163, 7695377.4920183625]
AUC                                                                                                                    
0.9611111111111112                                                                                                     
[632.7397222405715, 7768987.901437537, 4.311859275402941, 160.45649683900277, 9667779.018406456, 9087921.628429

D:\programy\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)

D:\programy\Anaconda\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



[424.4521496168387, 8319195.490534725, 7.939414591722823, 148.69174858577225, 7321635.510188403, 5826.651144454414, 2866656.4206394595]
[318.9300657535001, 2078634.0057047647, 422.7190861703999, 76.13186421571493, 6389257.99757855, 559388.0382012734, 967531.3790065167]
[398.05578616764933, 965127.4946628932, 284.83994849811944, 92.63414410731252, 7421793.018497104, 4394228.693544245, 6249075.782806657]
[352.80465532871415, 2669088.3046555487, 42.52599669769479, 222.69950287855306, 6758396.193200496, 888306.182026738, 3261231.7365908884]
[295.3575985888935, 5295192.075203415, 364.3075762189358, 444.5205632754592, 9880338.214844422, 4018323.313686598, 5680463.632933]
[49.30849834230414, 8963641.375906676, 0.6451704593086731, 354.0943782755237, 8001822.993259977, 1254744.8552508678, 6409868.869000269]
[669.1529376954484, 3054817.942439114, 23.461037481870815, 451.8991506843601, 8371790.117940014, 474341.71191193594, 8038965.018815139]
[442.4521319412638, 3173151.7839172515, 33.71282559352

D:\programy\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)

D:\programy\Anaconda\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



[335.10578181280255, 368827.43696876935, 143.54269945252898, 267.6691181713047, 9134708.242237754, 416845.6912860306, 3138279.589552654]
[74.03971084600309, 33846.10192195425, 343.8524217139242, 289.15143025313125, 7581951.491238579, 1064818.6872564212, 280703.817294647]
[266.0305546498891, 721554.8723516909, 483.35619080266565, 428.73589109623157, 7904724.756135503, 611833.6576990313, 2784482.7936718697]
[315.7868967053068, 383999.7810339472, 397.75972557399916, 252.0881088680385, 7819896.153291337, 359338.7678455263, 3520215.9365658644]
[213.55651507620843, 5851857.346556622, 99.37449984144811, 187.51325207224656, 5141315.613443314, 2457414.191087633, 8332263.814859004]
[117.31831585898948, 1858347.8267978814, 435.9700492665606, 474.6713251719964, 9244797.883700829, 1474876.2665041643, 8735130.408271315]
[489.2635992880279, 2316984.883079314, 499.91970723568477, 495.7442222234139, 8948567.176321635, 846073.7294534562, 8886948.66338098]
[115.6479573770763, 5513167.384720858, 78.801798

In [6]:
print(space_eval(space, best))

{'lambda1': 779.5700009689626, 'lambda2': 4830131.9980255505, 'lambda3': 312.1580702974475, 'lambda4': 134.63105864272865, 'lambda5': 3329905.077922123, 'lambda6': 59003.45150548895, 'lambda7': 6090330.444138394}


In [7]:
best_lambda_array = list(space_eval(space, best).values())

ids_multiclass = IDSOneVsAll(algorithm="RUSM")
ids_multiclass.fit(quant_dataframe, lambda_array=best_lambda_array, rule_cutoff=30)
auc = ids_multiclass.score_auc(quant_dataframe)

In [8]:
auc

0.9611111111111112

In [9]:
ids_multiclass.score_interpretability_metrics(quant_dataframe)

{'fraction_overlap': 0.03753086419753086,
 'fraction_classes': 1.0,
 'fraction_uncovered': 0.14320987654320985,
 'average_rule_width': 2.277777777777778,
 'ruleset_length': 6.0}

In [10]:
cba = CBA(support=0, confidence=0)
txns = TransactionDB.from_DataFrame(df)

cba.fit(txns)
cba.rule_model_accuracy(txns)

0.9407407407407408

In [11]:
df = pd.read_csv("../data/segment0.csv")
quant_dataframe = QuantitativeDataFrame(df)

ids_multiclass = IDSOneVsAll(algorithm="RUSM")
ids_multiclass.fit(quant_dataframe, lambda_array=best_lambda_array, rule_cutoff=50)
auc = ids_multiclass.score_auc(quant_dataframe)

auc-';°'

0.7567740901074235

In [ ]:
def objective(args):
    lambda_array = list(args.values())
    print(lambda_array)
    
    ids_multiclass = IDSOneVsAll(algorithm="RUSM")
    ids_multiclass.fit(quant_dataframe, lambda_array=lambda_array, rule_cutoff=30)
    
    metrics = ids_multiclass.score_interpretability_metrics(quant_dataframe)
    
    if not is_solution_interpretable(metrics):
        return 0
    
    auc = ids_multiclass.score_auc(quant_dataframe)
    print("AUC", auc)

    return -auc

space = {
    "lambda1": hp.uniform("l1", 0, 1000),
    "lambda2": hp.uniform("l2", 0, 10000000),
    "lambda3": hp.uniform("l3", 0, 500),
    "lambda4": hp.uniform("l4", 0, 500),
    "lambda5": hp.uniform("l5", 0, 10000000),
    "lambda6": hp.uniform("l6", 0, 10000000),
    "lambda7": hp.uniform("l7", 0, 10000000)
}

best = fmin(objective, space, algo=tpe.suggest, max_evals=500)

print(best)

[652.1517688857958, 9203529.891177068, 362.1448861373701, 334.85981287483975, 2729536.943221206, 8560488.263530202, 6236394.897953363]
  0%|                                                                          | 0/500 [00:00<?, ?trial/s, best loss=?]

D:\programy\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)

D:\programy\Anaconda\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



[532.3604827502271, 4599084.024118777, 324.7828639335177, 266.771735265762, 4881386.842201881, 5482306.634920353, 5376633.515823349]
[639.5488138255362, 9916540.353570009, 233.8197599706261, 449.23264403906086, 2227958.130621458, 5352683.450235693, 7640859.492372396]
[125.16016240387262, 9610788.497466125, 435.05508792775464, 35.16318721376172, 6168504.520121843, 1721679.860977925, 5964110.784077803]
[856.1363653226862, 5208693.96235933, 419.65586054654005, 230.25993119772082, 3376231.042693162, 4310241.967044583, 4179037.2885578196]
[899.7262913936803, 7596074.342321355, 111.16895023511353, 186.02706472757185, 3990787.0834798063, 9531594.04674724, 1456184.3794973728]
[358.108780951569, 5511303.396257774, 73.2014723158907, 109.6980449223891, 8362511.728903069, 7723060.041963535, 4164331.3566381023]
[909.512194490104, 3389636.9233002667, 313.14512228528554, 322.2510868447779, 6913858.5481052585, 1175410.5269593108, 2243957.0370007334]
[0.4324988936867946, 894006.5430888921, 111.11554261

[175.0326819817857, 7501325.160242845, 413.9394005579171, 445.3448345787462, 2708003.355252241, 185790.94363875967, 3363648.82735882]
[954.3118178135837, 6898999.774727123, 233.97772043305318, 307.5733697329649, 1361822.4214115764, 8660523.030750658, 1610987.7692419067]
[667.7481215396022, 5427641.288058732, 232.94436055697346, 345.03771021662504, 2456345.6483083796, 9372462.587027367, 1809504.0754237236]
[917.4242759223309, 2602153.254585025, 454.2755343266711, 290.57692235458535, 5919067.961369919, 7984586.783630852, 7002641.931713537]
[38.287901878321975, 4788830.987450316, 476.1210210369132, 323.7375838500887, 6647661.995219993, 4495570.252779115, 7234430.954266065]
[563.0801956677927, 6391127.880230173, 298.5938683681425, 345.7350557441001, 7833474.3255964145, 4144485.642313118, 6208846.2440676065]
[524.5114118035353, 5366765.846427033, 284.88890339341293, 386.6623120045391, 7043529.177017278, 2342701.489946878, 5058665.485734254]
[383.00963530165404, 6108732.855754371, 121.255322